In [ ]:
import pandas as pd
import sqlite3

In [ ]:
import sqlite3
conn = sqlite3.connect("/content/drive/MyDrive/Colab Notebooks/anekdot-ru.db")
df = pd.read_sql_query("SELECT text FROM anecdot WHERE date BETWEEN '2019-01-01' and '2020-01-31'", conn) # WHERE date BETWEEN '2020-01-01' and '2020-01-31'
# Verify that result of SQL query is stored in the dataframe
print(df.head())

                                                text
0       1 января — День святого рассола и минералки.
1  Уровень тишины 1 января: слышно, как у соседей...
2  В 8 часов утра 1 января собачники могли бы зах...
3  - А я вот загадал на Новый Год желание, чтобы ...
4  Новый год без ели – что Дед Мороз без хмеля!\n...


In [ ]:
f = open('anecdot.txt', 'w')
for i in df['text']:
    f.write(i + '\n')

In [ ]:
df.info

<bound method DataFrame.info of                                                     text
0           1 января — День святого рассола и минералки.
1      Уровень тишины 1 января: слышно, как у соседей...
2      В 8 часов утра 1 января собачники могли бы зах...
3      - А я вот загадал на Новый Год желание, чтобы ...
4      Новый год без ели – что Дед Мороз без хмеля!\n...
...                                                  ...
26625  У темнокожих красоток все  дыры черные и косми...
26626  Все планы мирного урегулирования израильско-па...
26627                    Все жёны хороши, кроме скучной.
26628  Когда в семье педагогов появляется ребёнок, вс...
26629  А вот интересно, если в Китае есть город Ухань...

[26630 rows x 1 columns]>

In [ ]:
!pip install transformers 

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Загружаем модель
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

     |████████████████████████████████| 4.2 MB 5.4 MB/s 
     |████████████████████████████████| 6.6 MB 35.4 MB/s 
     |████████████████████████████████| 84 kB 3.2 MB/s 
     |████████████████████████████████| 596 kB 48.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

In [ ]:
train_path = '/content/anecdot.txt'

In [ ]:
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                mlm=False)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(  
    output_dir="./finetuned", # The output directory
    overwrite_output_dir=True, # Overwrite the content of the output dir
    num_train_epochs=25, # number of training epochs
    per_device_train_batch_size=20, # batch size for training
    per_device_eval_batch_size=20,  # batch size for evaluation
    warmup_steps=10, # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=16, # to make "virtual" batch size larger
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5), None)
)

In [ ]:
trainer.train()
torch.save(model,'/content/drive/MyDrive/Colab Notebooks/anekd.pth')

***** Running training *****
  Num examples = 1395
  Num Epochs = 25
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 320
  Gradient Accumulation steps = 16
  Total optimization steps = 100


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




In [ ]:
torch.save(model,'/content/drive/MyDrive/Colab Notebooks/anekd.pth')

NameError: ignored

In [ ]:
model = torch.load('/content/drive/MyDrive/Colab Notebooks/anekdyear.pth')

In [ ]:
text = "Из практики сисадмина"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=100,
                        )

generated_text = list(map(tokenizer.decode, out))[0]

print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Из практики сисадмина:
Если компьютерная игра вышла из строя, то в ней нет ничего нового.
Если в жизни есть что-то ценное, то, наверное, ничего и нет.
- Дорогой, я тебя очень прошу, не звони больше мне больше! Я не люблю тебя. Ты меня бросил!
- Да, что случилось, что случилось?
- Я не могу позвонить тебе больше.
- Почему?
- Ну, почему?
- Я, наверное
